This notebook shows how to run playoff scenarios prior to the end of regular season,
and identify how each team might win their division, whether they "control their
own destiny" etc. This example is scoped to the last two weeks of regular
season (weeks 17 and 18). If you're running the analysis after the end of
the season then these scores will be in the database, but you can clear them
and replace them with your own assumptions as shown below.

In the future I hope to do similar notebooks for wilcard slots and home-field advantage

In [1]:
from nfl import NFL
import pandas as pd
import numpy as np
import time

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display

nfl = NFL().load()

In [2]:
# For reference, print conference standings not counting weeks 17 and 18
weeks = [17, 18]
nfl.clear(weeks)
nfl('NFC')

NFC
            div overall                    division                conference  \
                    win loss tie       pct      win loss tie   pct        win   
team                                                                            
PHI    NFC-East      11    4   0  0.733333        4    1   0   0.8          7   
DAL    NFC-East      10    5   0  0.666667        4    1   0   0.8          7   
NYG    NFC-East       5   10   0  0.333333        2    3   0   0.4          4   
WAS    NFC-East       4   11   0  0.266667        0    5   0   0.0          2   
DET   NFC-North      11    4   0  0.733333        3    2   0   0.6          7   
MIN   NFC-North       7    8   0  0.466667        2    2   0   0.5          6   
GB    NFC-North       7    8   0  0.466667        2    2   0   0.5          5   
CHI   NFC-North       6    9   0       0.4        2    3   0   0.4          5   
TB    NFC-South       8    7   0  0.533333        3    1   0  0.75          6   
ATL   NFC-South       7    8   0  0.466667        3    2   0   0.6          4   
NO    NFC-South       7    8   0  0.466667        2    2   0   0.5          4   
CAR   NFC-South       2   13   0  0.133333        1    4   0   0.2          1   
SF     NFC-West      11    4   0  0.733333        5    0   0   1.0          9   
STL    NFC-West       8    7   0  0.533333        4    1   0   0.8          6   
SEA    NFC-West       8    7   0  0.533333        1    4   0   0.2          6   
ARI    NFC-West       3   12   0       0.2        0    5   0   0.0          2   

                         
     loss tie       pct  
team                     
PHI     3   0       0.7  
DAL     3   0       0.7  
NYG     6   0       0.4  
WAS     8   0       0.2  
DET     3   0       0.7  
MIN     4   0       0.6  
GB      5   0       0.5  
CHI     5   0       0.5  
TB      4   0       0.6  
ATL     6   0       0.4  
NO      6   0       0.4  
CAR    10   0  0.090909  
SF      1   0       0.9  
STL     4   0       0.6  
SEA     5   0  0.545455  
ARI     8   0       0.2

In [3]:
# Specify the division and week range to run
div = 'NFC-South'
weeks = [17, 18]

nfl.reload()
nfl.clear(weeks)
nfl(div)

NFC-South
     overall                    division               
         win loss tie       pct      win loss tie   pct
team                                                   
TB         8    7   0  0.533333        3    1   0  0.75
ATL        7    8   0  0.466667        3    2   0   0.6
NO         7    8   0  0.466667        2    2   0   0.5
CAR        2   13   0  0.133333        1    4   0   0.2

In [4]:
# limit scope to teams still in contention
st = nfl(div).standings
teams = set(st[st[('overall','win')] >= st.iloc[0][('overall','win')] - len(weeks)].index)

# print the relevant schedule for reference. It's helpful to know
# who plays who
nfl.schedule(teams, weeks, by='game')

at hscore ascore
week ht                    
17   CHI  ATL   None   None
     TB    NO   None   None
18   CAR   TB   None   None
     NO   ATL   None   None

In [5]:
# iterate over all possible outcomes and count the number
# of times each team wins the division.
# this can take 30 seconds or more in fast mode for a 2-week span

results = pd.DataFrame(columns=pd.MultiIndex.from_product([weeks, teams], names=['week','team']))
results.index.name = 'scenario'
results[('result','outcome')] = np.nan
results[('result','rule')] = np.nan

# calculate the number of scenarios to set up the progress bar
# it's the number of potential outcomes (win,loss.tie = 3)
# to the power of the number of relevant games (4 in this case)
# pow(3, 4) = 81

bar = IntProgress(min=0, max=3 ** len(nfl.schedule(teams, weeks, by='game')))
display(bar)

start = time.time()
for elem in nfl.scenarios(weeks, teams):
    nfl.clear(weeks)
    nfl.set(elem)

    sch = nfl.schedule(teams, weeks)

# NB: if you are only interested in specific scenarios, e.g.
# ones where a particuar team wins all their remaining games
# you can typically save a lot of time by querying the resulting
# schedule for the current scenario at this point and avoid
# the call to tiebreaks, which even in fast mode can be
# very time consuming
    
    t = nfl.tiebreaks(teams, fast=True)

    z = len(results)
    results.loc[z] = sch['wlt']
    results.loc[z, ('result','outcome')] = t.index[0]
    if len(t) > 1:
        results.loc[z, ('result','rule')] = t.iloc[1]
    else:
        results.loc[z, ('result','rule')] = 'overall'

    bar.value += 1

bar.layout.display = 'none'
print('Elapsed time: {}'.format(time.time() - start))
results

IntProgress(value=0, max=81)

Elapsed time: 17.120001077651978


week        17               18              result         
team        NO   TB   ATL    NO    TB   ATL outcome     rule
scenario                                                    
0         loss  win  loss   win  loss  loss      TB  overall
1         loss  win  loss  loss  loss   win      TB  overall
2         loss  win  loss   tie  loss   tie      TB  overall
3         loss  win  loss   win   win  loss      TB  overall
4         loss  win  loss  loss   win   win      TB  overall
...        ...  ...   ...   ...   ...   ...     ...      ...
76         tie  tie   tie  loss   win   win      TB  overall
77         tie  tie   tie   tie   win   tie      TB  overall
78         tie  tie   tie   win   tie  loss      TB  overall
79         tie  tie   tie  loss   tie   win      TB  overall
80         tie  tie   tie   tie   tie   tie      TB  overall

[81 rows x 8 columns]

In [6]:
# This shows how many scenarios result in each team winning the division
results.groupby(('result','outcome')).count()

week               17          18         result
team               NO  TB ATL  NO  TB ATL   rule
(result, outcome)                               
ATL                 8   8   8   8   8   8      8
NO                  8   8   8   8   8   8      8
TB                 65  65  65  65  65  65     65

In [7]:
# this will report all scenarios where the specified team wins
# their remaining games, answering the question whether they
# "control their own destiny." If so then the result will invariably
# be that the team wins the division, as shown in the 2nd column from the right

# Note that if a rule shows as conference|overall-rank or any form of netpoints,
# the analysis is not necessarily valid because the scenarios are based on
# outcomes only without specifying point spreads

results[(results.xs('NO',level=1,axis=1) == 'win').all(axis=1)]

week       17               18              result              
team       NO    TB   ATL   NO    TB   ATL outcome          rule
scenario                                                        
9         win  loss  loss  win  loss  loss      NO       overall
12        win  loss  loss  win   win  loss      TB  common-games
15        win  loss  loss  win   tie  loss      NO       overall
36        win  loss   win  win  loss  loss      NO       overall
39        win  loss   win  win   win  loss      TB  common-games
42        win  loss   win  win   tie  loss      NO       overall
63        win  loss   tie  win  loss  loss      NO       overall
66        win  loss   tie  win   win  loss      TB  common-games
69        win  loss   tie  win   tie  loss      NO       overall